In [1]:
%load_ext soup

Monkey patch BeautifulSoup with custom rendering
See `configure_ipython_beautifulsoup?` for configuration information
Push 'BeautifulSoup' from 'bs4' into current context
Push 'urlopen' from 'urllib.request' into current context
Push 'p' shortcut into current context
Push 'requests' into current context


In [2]:
url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247"
configure_ipython_beautifulsoup(show_html=True, show_css=True, show_js=False)

In [3]:
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
request_codes = {}
for input in soup.find_all('input'):
    for element in input.find_all('option'):
        request_codes[element.text] = (input['name'], element['value'])

In [5]:
arguments = {}
arguments["ww_x_GPS"] = "-1"
arguments["ww_i_reportModel"] = "133685247"
arguments["ww_i_reportModelXsl"] = "133685270"
arguments[request_codes["Informatique"][0]] = request_codes["Informatique"][1]
arguments[request_codes["2007-2008"][0]] = request_codes["2007-2008"][1]
arguments[request_codes["Bachelor semestre 1"][0]] = request_codes["Bachelor semestre 1"][1]
arguments["ww_x_HIVERETE"] = "null"

{'ww_i_reportModelXsl': 133685270,
 'ww_x_GPS': '-1',
 'ww_x_HIVERETE': 'null',
 'zz_x_PERIODE_ACAD': '978181',
 'zz_x_PERIODE_PEDAGO': '249108',
 'zz_x_UNITE_ACAD': '249847'}

In [ ]:
url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?"
url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=71297531&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=978181&ww_x_PERIODE_PEDAGO=249108&ww_x_HIVERETE=null"
#r = requests.get(url, params=arguments)
r = requests.get(url)
print(r.text)